# Cross Validation

## 課題J-1

### 交差検証とは

データをいくつかに分割して、1つをテストデータとし残りを訓練データとしてモデルを訓練し評価する手法。  
例えばデータをA,B,Cの3つに分割したとすると、最初にAをテストデータとして残りのB,Cを訓練データとして評価を行う。次にBをテストデータとして同じく評価を行う。Cについても同様である。
このように訓練データとテストデータを交差させながら精度を評価することである。

ホールドアウトはデータを訓練データとテストデータに分割するが、同じテストデータで何度もフィッティングを行う。  
交差検証はテストデータと訓練データを入れ替えるということを行う。これが'交差'を表している。

同じテストデータでフィッティングをし続けてしまうとそのテストデータにあったモデルが出来上がってしまう。それを避けるために、テストデータと訓練データを変えることによって、未知のデータが入力されたときに高い性能を出すことができる。

### 交差検証の実装

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings('ignore')

%matplotlib inline

### データの準備

In [2]:
from sklearn.datasets import load_iris

iris = load_iris()

In [31]:
X = iris.data
y = iris.target
X.shape

(150, 4)

In [5]:
df = pd.DataFrame(X, columns=iris.feature_names)

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


### データの分割

In [39]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=3, shuffle=True, random_state=0)
kf.get_n_splits(df)

for train_index, test_index in kf.split(df):
    X_train , X_test = df.iloc[train_index], df.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    logreg = LogisticRegression()
    
    logreg.fit(X_train, y_train)
    
    print(accuracy_score(y_true=y_test,
              y_pred=logreg.predict(X_test)))


0.9
0.96
0.96


### StratifiedKFold

StratifiedKFoldはクラス間の訓練データとテストデータの割合を考慮したデータ分割を行う。
<br>  
Aクラス=50、Bクラス＝50　計100個がこの順で並んでいるデータがあるとする。  
単純に3:1で分けるとすると、Aが50とBが25の計75個の訓練データ、残りBが25のテストデータに分けられてしまう。この場合、テストデータにはBクラスしか含まれない。  
これを防ぐためにはAクラスから3:1、Bクラスから3:1の割合(訓練:テスト)で分割できれば良い。 
<br>  
この操作を行うのがStratifiedKFoldである。

In [42]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
skf.get_n_splits(df, y)

for train_index, test_index in skf.split(df, y):
    X_train , X_test = df.iloc[train_index], df.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    logreg = LogisticRegression()
    
    logreg.fit(X_train, y_train)
    
    print(accuracy_score(y_true=y_test,
              y_pred=logreg.predict(X_test)))

0.9215686274509803
0.9803921568627451
0.9583333333333334


### GroupKFold